# Robust Portfolio

This tutorial shows how to construct and solve the Robust Portfolio problem, introduced by Bertsimas and Sim 2004 [1] (https://pubsonline.informs.org/doi/abs/10.1287/opre.1030.0065)
We show to how solve the simple portfolio problem (Section 6.2). Using the same formulation as in [1], the problem can be formulated as follows:

$$
\begin{array}{ll}
\text{maximize} & \mathbf{p}^T \mathbf{x} \\
\text{subject to} & \mathbf{1}^T\mathbf{x} = 1 \\
                  & \mathbf{x}\geq \mathbf{0} \\
                  & \mathbf{p_e}-\pmb{\sigma} \leq \mathbf{p} \leq \mathbf{p_e}+\pmb{\sigma}

\end{array}
$$

where there are $n$ stocks, the unknown return of the stocks is denoted by $\mathbf{p}$, and $\mathbf{x}$ denotes the fraction fo wealth invested in the stocks. The expected return of the stocks are denoted by $\mathbf{p_e}$, and their uncertainties are captured by $\pmb{\sigma}$.

In [1]:
import cvxpy as cp
import numpy as np
import lropt

np.random.seed(seed=1234)

We start by defining the relevant constants. We use the same values as given in the example in Section 6.2 of [1].

In [2]:
n = 150 #Number of stocks
i = np.arange(1, n+1) #Indices vector
p_e = 1.15 + i*0.05/150 #Mean returns vector
sigma = np.array(0.05/450 * (2*i*n*(n+1))**0.5) #Uncertainty vector

The uncertain parameter $\mathbf{p}$ is formulated using LROPT in the block below. We use the box uncertainty set, which is defined as follows:

Box uncertainty: $\{ u = Az+b \mid\| z\|_\infty \leq \rho  \}$ 

- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We define an uncertain parameter $\mathbf{p}$ using the Box definition with $\rho=1,\mathbf{A}=\text{diag}\left( \pmb{\sigma}\right), \mathbf{b}=\mathbf{p_e}$.

In [3]:
#Define a helper uncertainty variable u
uncertainty_set = lropt.Box(rho=1, a=np.diag(sigma), b=p_e)
p = lropt.UncertainParameter(n, uncertainty_set=uncertainty_set)
x = cp.Variable(n) #Optimization variable

#Define and solve the problem
objective = cp.Maximize(p@x)
constraints = [
                cp.sum(x)==1,
                x>=0,
              ]
prob = lropt.RobustProblem(objective=objective, constraints=constraints)
prob.solve(solver="CLARABEL")


/Users/mj5676/Desktop/miniconda3/envs/lropt_v3/lib/python3.12/site-packages/scipy/sparse/_index.py:134: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/Users/mj5676/Desktop/miniconda3/envs/lropt_v3/lib/python3.12/site-packages/cvxpy/utilities/torch_utils.py:61: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:643.)
  return torch.sparse.FloatTensor(i, v, torch.Size(value_coo.shape)).to(dtype)
